In [1]:
import json
import os
import warnings

import juliacall

import pymatgen
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.core import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter

from tqdm.notebook import tqdm

print("Using pymatgen version:", pymatgen.core.__version__)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Using pymatgen version: 2024.6.10


In [2]:
jl = juliacall.newmodule("NotebookModule")
jl.seval("using CrystalNets")
jl.CrystalNets.toggle_export(False)

print("Using Julia version:", jl.seval("VERSION"))
print("Running from directory:", jl.seval("Sys.BINDIR"))

Using Julia version: 1.10.4
Running from directory: /opt/julia-1.10.4/bin


In [3]:
with open(os.path.expanduser("~/.mpapikey"), "r") as f:
    apikey = f.read().strip()

In [7]:
crystallnn = CrystalNN()

try:
    os.mkdir("cif")
except FileExistsError as e:
    pass

def make_labels_unique(struct):
    from collections import Counter
    
    labels = Counter(site.label for site in struct.sites)
    counter = {}
    for i, site in enumerate(struct.sites):
        label = site.label
        if labels[label] > 1 or label.isalpha():
            c = counter.get(label, 0)
            site.label = f"{label}{c}" if label.isalpha() else f"{label}_{c}"
            c = c + 1
            counter[label] = c


def jimage_to_site_symmetry(jimage):
    i, j, k = jimage
    return f"1_{5+i}{5+j}{5+k}"


def writeToCifFile(bonded_struct, file):
    cif_writer = CifWriter(bonded_struct.structure)
    cif = str(cif_writer)
    cif += """loop_
_geom_bond_atom_site_label_1
_geom_bond_atom_site_label_2
_geom_bond_distance
_geom_bond_site_symmetry_2
"""

    for n, site in enumerate(bonded_struct.structure.sites):
        for connected in bonded_struct.get_connected_sites(n):
            # Make sure we only output each bond once
            if site.label <= connected.site.label:
                cif += f"{site.label} {connected.site.label} {connected.dist:.6f} {jimage_to_site_symmetry(connected.jimage)}\n"

    file.write(cif)


def analyse_bonding(mat):
    # Magnetic moments trigger a bug in CifWriter, so we remove them here
    # https://github.com/materialsproject/pymatgen/issues/3772
    if "magmom" in mat.structure.site_properties:
        mat.structure.remove_site_property("magmom")

    # All labels should be unique (otherwise bond specifications will fail)
    make_labels_unique(mat.structure)
    labels = [site.label for site in mat.structure.sites]
    if len(labels) != len(set(labels)):
        raise ValueError("labels are not unique in structure")

    while True:
        bonded_struct = None
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            try:
                bonded_struct = crystallnn.get_bonded_structure(mat.structure)
            except Exception as e:
                print(f"CrystalNN failed for {mat.material_id}: {e}")
                return None

        coord = set()
        bonds = set()
        rerun_flag = False
        for n, site in enumerate(bonded_struct.structure.sites):
            neighbors = bonded_struct.get_connected_sites(n)
            # We cannot use get_coordination_of_site() because of a bug:
            # https://github.com/materialsproject/pymatgen/issues/3888#issuecomment-2232571072
            coord.add((str(site.specie), len(neighbors)))
            for connected in neighbors:
                if site.specie <= connected.site.specie:
                    bonds.add((str(site.specie), str(connected.site.specie)))
                if site.label == connected.site.label:
                    # If an atom is bonded to its own image, then CrystalNets will not like it
                    # and we need to make rerun on a supercell
                    rerun_flag = True

        if rerun_flag:
            # If we found an atom bonded to itself, double the cell and restart
            mat.structure.make_supercell(2, in_place=True)
            make_labels_unique(mat.structure)
        else:
            # Otherwise we're done
            break

    with open(f"cif/{mat.material_id}.cif", "w") as f:
        writeToCifFile(bonded_struct, f)

    return {"material_id": str(mat.material_id),
            "formula_pretty": mat.formula_pretty,
            "nelements": mat.nelements,
            "coordination": coord,
            "bonds": bonds}

In [5]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-3934"],
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )

    assert len(mp_data) == 1
    res = analyse_bonding(mp_data[0])
    print(res)

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

{'material_id': 'mp-3934', 'formula_pretty': 'Cu3PS4', 'nelements': 3, 'coordination': {('Cu', 4), ('P', 4), ('S', 4)}, 'bonds': {('Cu', 'S'), ('P', 'S')}}


In [6]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )
    print("Number of materials found:", len(mp_data))
    print("Database version", mpr.get_database_version())

Retrieving SummaryDoc documents:   0%|          | 0/153235 [00:00<?, ?it/s]

Number of materials found: 155361
Database version 2023.11.1


In [7]:
data = {}

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="No Pauling electronegativity")
    for mat in tqdm(mp_data, smoothing=0.01, mininterval=1):
        x = analyse_bonding(mat)
        if x is not None:
            data[mat.material_id] = x

print("Number of systems analyzed:", len(data))

  0%|          | 0/155361 [00:00<?, ?it/s]

CrystalNN failed for mp-994911: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1210439: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1247838: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1213668: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1215144: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1180797: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1247813: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1212347: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1212578: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1214815: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1215160: No Voronoi neighbors found for site - 

In [20]:
data["mp-1206677"]

{'material_id': 'mp-1206677',
 'formula_pretty': 'Rb2O2',
 'nelements': 2,
 'coordination': {('O', 6), ('Rb', 6)},
 'bonds': {('Rb', 'O')}}

In [8]:
crystalnets_options = jl.CrystalNets.Options(structure=jl.StructureType.Auto, clusterings=[jl.Clustering.EachVertex], bonding=jl.Bonding.Input, split_O_vertex=False)

def analyse_topology(material_id):
    res = jl.determine_topology(f"cif/{material_id}.cif", crystalnets_options)
    topologies = [(mult, jl.ndims(net[jl.Clustering.EachVertex].genome), str(net[jl.Clustering.EachVertex])) for net, mult in res]
    return topologies

In [26]:
analyse_topology("mp-1206677")

[(1, 3, 'pcu')]

In [32]:
jl.CrystalNets.toggle_export(False)
jl.CrystalNets.toggle_warning(False)

for material_id in tqdm(data.keys(), smoothing=0.01, mininterval=1):
    topo = analyse_topology(material_id)
    data[material_id]["topology"] = topo

  0%|          | 0/155345 [00:00<?, ?it/s]

spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).


In [33]:
data["mp-1206677"]

{'material_id': 'mp-1206677',
 'formula_pretty': 'Rb2O2',
 'nelements': 2,
 'coordination': {('O', 6), ('Rb', 6)},
 'bonds': {('Rb', 'O')},
 'topology': [(1, 3, 'pcu')]}

In [43]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

with open("topo_data.json", "w") as file:
    json.dump(data, file, cls=SetEncoder)

# Debugging

In [9]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-2647057"],
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )

    assert len(mp_data) == 1
    res = analyse_bonding(mp_data[0])
    print(res)
    print(analyse_topology("mp-2647057"))

# Bug filed at https://github.com/materialsproject/pymatgen/issues/3888

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

{'material_id': 'mp-2647057', 'formula_pretty': 'Co', 'nelements': 1, 'coordination': {('Co', 8)}, 'bonds': {('Co', 'Co')}}
[(1, 3, 'bcu')]


In [25]:
structure = mp_data[0].structure
bonded_struct = crystallnn.get_bonded_structure(structure)
bonded_struct

/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4146: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:3939: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


Structure Graph
Structure: 
Structure Summary
Lattice
    abc : 2.428889488469988 2.428889488469988 2.428889488469988
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 11.030656874268672
      A : -1.40232 1.40232 1.40232
      B : 1.40232 -1.40232 1.40232
      C : 1.40232 1.40232 -1.40232
    pbc : True True True
PeriodicSite: Co0 (Co) (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
Graph: bonds
from    to  to_image    
----  ----  ------------
   0     0  (1, 1, 1)   
   0     0  (1, 0, 0)   
   0     0  (0, 0, 1)   
   0     0  (0, 1, 0)   

In [11]:
crystallnn.get_cn(structure, 0)

8

In [12]:
crystallnn.get_cn_dict(structure, 0)

{'Co': 8}

In [13]:
from pymatgen.analysis.graphs import StructureGraph

struct_graph = StructureGraph.from_empty_graph(structure, name="bonds")
for idx, neighbors in enumerate(crystallnn.get_all_nn_info(structure)):
    for neighbor in neighbors:
        print(neighbor["site_index"], neighbor["image"])
        struct_graph.add_edge(from_index=idx, from_jimage=(0, 0, 0), to_index=neighbor["site_index"], to_jimage=neighbor["image"])

/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/graphs.py:437: UserWarning: Trying to add an edge that already exists from site 0 to site 0 in (1, 1, 1).
  warnings.warn(
/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/graphs.py:437: UserWarning: Trying to add an edge that already exists from site 0 to site 0 in (1, 0, 0).
  warnings.warn(
/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/graphs.py:437: UserWarning: Trying to add an edge that already exists from site 0 to site 0 in (0, 1, 0).
  warnings.warn(
/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/graphs.py:437: UserWarning: Trying to add an edge that already exists from site 0 to site 0 in (0, 0, 1).
  warnings.warn(


0 [-1. -1. -1.]
0 [1. 1. 1.]
0 [1. 0. 0.]
0 [0. 0. 1.]
0 [-1.  0.  0.]
0 [ 0. -1.  0.]
0 [0. 1. 0.]
0 [ 0.  0. -1.]


In [26]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-2647057"],
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )

    assert len(mp_data) == 1
    mp_data[0].structure.make_supercell(2)
    res = analyse_bonding(mp_data[0])
    print(res)
    print(analyse_topology("mp-2647057"))

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

{'material_id': 'mp-2647057', 'formula_pretty': 'Co', 'nelements': 1, 'coordination': {('Co', 8)}, 'bonds': {('Co', 'Co')}}
[(1, 3, 'bcu')]


In [16]:
bonded_struct.get_connected_sites(0)

[ConnectedSite(site=PeriodicSite: Co (1.402, 1.402, -1.402) [0.0, 0.0, 1.0], jimage=(0, 0, 1), index=0, weight=1, dist=2.428889488469988),
 ConnectedSite(site=PeriodicSite: Co (-1.402, 1.402, -1.402) [0.0, -1.0, 0.0], jimage=(0, -1, 0), index=0, weight=1, dist=2.428889488469988),
 ConnectedSite(site=PeriodicSite: Co (-1.402, -1.402, -1.402) [-1.0, -1.0, -1.0], jimage=(-1, -1, -1), index=0, weight=1, dist=2.428889488469988),
 ConnectedSite(site=PeriodicSite: Co (1.402, -1.402, 1.402) [0.0, 1.0, 0.0], jimage=(0, 1, 0), index=0, weight=1, dist=2.428889488469988),
 ConnectedSite(site=PeriodicSite: Co (-1.402, 1.402, 1.402) [1.0, 0.0, 0.0], jimage=(1, 0, 0), index=0, weight=1, dist=2.428889488469988),
 ConnectedSite(site=PeriodicSite: Co (1.402, -1.402, -1.402) [-1.0, 0.0, 0.0], jimage=(-1, 0, 0), index=0, weight=1, dist=2.428889488469988),
 ConnectedSite(site=PeriodicSite: Co (-1.402, -1.402, 1.402) [0.0, 0.0, -1.0], jimage=(0, 0, -1), index=0, weight=1, dist=2.428889488469988),
 Connected

In [14]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-2647057"],
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )

structure = mp_data[0].structure
bonded_struct = crystallnn.get_bonded_structure(structure)
bonded_struct

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Structure Graph
Structure: 
Structure Summary
Lattice
    abc : 2.428889488469988 2.428889488469988 2.428889488469988
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 11.030656874268672
      A : -1.40232 1.40232 1.40232
      B : 1.40232 -1.40232 1.40232
      C : 1.40232 1.40232 -1.40232
    pbc : True True True
PeriodicSite: Co (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
Graph: bonds
from    to  to_image    
----  ----  ------------
   0     0  (1, 1, 1)   
   0     0  (1, 0, 0)   
   0     0  (0, 0, 1)   
   0     0  (0, 1, 0)   

# Previous version of the code

In [1]:
import json
import os
import tempfile
import warnings

import juliacall

import pymatgen
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.core import Lattice, Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter

from tqdm.notebook import tqdm

print("Using pymatgen version:", pymatgen.core.__version__)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Using pymatgen version: 2024.6.10


In [2]:
jl = juliacall.newmodule("NotebookModule")
jl.seval("using CrystalNets")
jl.CrystalNets.toggle_export(False)

print("Using Julia version:", jl.seval("VERSION"))
print("Running from directory:", jl.seval("Sys.BINDIR"))

Using Julia version: 1.10.3
Running from directory: /opt/julia-1.10.3/bin


In [3]:
with open(os.path.expanduser("~/.mpapikey"), "r") as f:
    apikey = f.read().strip()

In [4]:
crystallnn = CrystalNN()
crystalnets_options = jl.CrystalNets.Options(structure=jl.StructureType.Auto, clusterings=[jl.Clustering.AllNodes], bonding=jl.Bonding.Input)


def make_labels_unique(struct):
    from collections import Counter
    
    labels = Counter(site.label for site in struct.sites)
    counter = {}
    for i, site in enumerate(struct.sites):
        label = site.label
        if labels[label] > 1 or label.isalpha():
            c = counter.get(label, 0)
            site.label = f"{label}{c}" if label.isalpha() else f"{label}_{c}"
            c = c + 1
            counter[label] = c


def jimage_to_site_symmetry(jimage):
    i, j, k = jimage
    return f"1_{5+i}{5+j}{5+k}"


def writeToCifFile(bonded_struct, file):
    cif_writer = CifWriter(bonded_struct.structure)
    cif = str(cif_writer)
    cif += """loop_
_geom_bond_atom_site_label_1
_geom_bond_atom_site_label_2
_geom_bond_distance
_geom_bond_site_symmetry_2
"""

    for n, site in enumerate(bonded_struct.structure.sites):
        for connected in bonded_struct.get_connected_sites(n):
            # Make sure we only output each bond once
            if site.label < connected.site.label:
                cif += f"{site.label} {connected.site.label} {connected.dist:.6f} {jimage_to_site_symmetry(connected.jimage)}\n"

    file.write(cif.encode())


def analyse_topology(struct):
    # All labels should be unique (otherwise bond specifications will fail)
    labels = [site.label for site in struct.sites]
    if len(labels) != len(set(labels)):
        raise ValueError("labels are not unique in structure")

    bonded_struct = None
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            bonded_struct = crystallnn.get_bonded_structure(struct)
        except Exception as e:
            print(f"CrystalNN failed for {mat.material_id}: {e}")

    coord = set()
    bonds = set()
    for n, site in enumerate(bonded_struct.structure.sites):
        coord.add((str(site.specie), bonded_struct.get_coordination_of_site(n)))
        for connected in bonded_struct.get_connected_sites(n):
            if site.specie <= connected.site.specie:
                bonds.add((str(site.specie), str(connected.site.specie)))

    with tempfile.NamedTemporaryFile(suffix=".cif", delete=False) as f:
        writeToCifFile(bonded_struct, f)
        f.close()
        res = jl.determine_topology(f.name, crystalnets_options)
        os.unlink(f.name)
        topologies = [(mult, jl.ndims(net[jl.Clustering.AllNodes].genome), str(net[jl.Clustering.AllNodes])) for net, mult in res]
        return {"topology": topologies, "coordination": coord, "bonds": bonds}

In [5]:
with MPRester(apikey) as mpr:
    structure = mpr.get_structure_by_material_id("mp-3934")
    make_labels_unique(structure)

    # Magnetic moments trigger a bug in CifWriter, so we remove them here
    # https://github.com/materialsproject/pymatgen/issues/3772
    if "magmom" in structure.site_properties:
        structure.remove_site_property("magmom")

    topo = analyse_topology(structure)
    print(topo)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

{'topology': [(1, 3, 'lon')], 'coordination': {('P', 4), ('S', 4), ('Cu', 4)}, 'bonds': {('P', 'S'), ('Cu', 'S')}}


In [6]:
def analyze_structure(mat):
    make_labels_unique(mat.structure)

    # Magnetic moments trigger a bug in CifWriter, so we remove them here
    # https://github.com/materialsproject/pymatgen/issues/3772
    if "magmom" in mat.structure.site_properties:
        mat.structure.remove_site_property("magmom")

    try:
        topo = analyse_topology(mat.structure)
        topo["material_id"] = mat.material_id
        topo["formula_pretty"] = mat.formula_pretty
        topo["nelements"] = mat.nelements
        return topo
    except Exception as e:
        print(f"Error with {mat.material_id}: {e}")

In [7]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )
    print("Number of materials found:", len(mp_data))
    print("Database version", mpr.get_database_version())

Retrieving SummaryDoc documents:   0%|          | 0/153235 [00:00<?, ?it/s]

Number of materials found: 155361
Database version 2023.11.1


In [9]:
jl.CrystalNets.toggle_export(False)
jl.CrystalNets.toggle_warning(False)
warnings.filterwarnings('ignore')

res = {}
for mat in tqdm(mp_data):
    res[mat.material_id] = analyze_structure(mat)

print("Number of systems analyzed:", len(res))

  0%|          | 0/155361 [00:00<?, ?it/s]

CrystalNN failed for mp-994911: No Voronoi neighbors found for site - try increasing cutoff
Error with mp-994911: 'NoneType' object has no attribute 'structure'


spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).


CrystalNN failed for mp-1210439: No Voronoi neighbors found for site - try increasing cutoff
Error with mp-1210439: 'NoneType' object has no attribute 'structure'


spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).


...

CrystalNN failed for mp-1182772: No Voronoi neighbors found for site - try increasing cutoff
Error with mp-1182772: 'NoneType' object has no attribute 'structure'


spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).

CrystalNN failed for mp-1212042: No Voronoi neighbors found for site - try increasing cutoff
Error with mp-1212042: 'NoneType' object has no attribute 'structure'


spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).

Number of systems analyzed: 155361


In [10]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

with open("topo_data.json", "w") as file:
    json.dump(res, file, cls=SetEncoder)

In [6]:
with MPRester(apikey) as mpr:
    structure = mpr.get_structure_by_material_id("mp-1206677")
    make_labels_unique(structure)

    # Magnetic moments trigger a bug in CifWriter, so we remove them here
    # https://github.com/materialsproject/pymatgen/issues/3772
    if "magmom" in structure.site_properties:
        structure.remove_site_property("magmom")

    topo = analyse_topology(structure)
    print(topo)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

{'topology': [(1, 3, 'UNKNOWN 3 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 2 0 1 1 1 2 1 1 1 1 2 2 1')], 'coordination': {('Rb', 6), ('O', 6)}, 'bonds': {('Rb', 'O')}}


In [7]:
def exportCif(struct):
    # All labels should be unique (otherwise bond specifications will fail)
    labels = [site.label for site in struct.sites]
    if len(labels) != len(set(labels)):
        raise ValueError("labels are not unique in structure")

    bonded_struct = None
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            bonded_struct = crystallnn.get_bonded_structure(struct)
        except Exception as e:
            print(f"CrystalNN failed for {mat.material_id}: {e}")

    coord = set()
    bonds = set()
    for n, site in enumerate(bonded_struct.structure.sites):
        coord.add((str(site.specie), bonded_struct.get_coordination_of_site(n)))
        for connected in bonded_struct.get_connected_sites(n):
            if site.specie <= connected.site.specie:
                bonds.add((str(site.specie), str(connected.site.specie)))

    with tempfile.NamedTemporaryFile(suffix=".cif", delete=False) as f:
        writeToCifFile(bonded_struct, f)
        f.close()
        res = jl.determine_topology(f.name, crystalnets_options)
        print(f.name)
        topologies = [(mult, jl.ndims(net[jl.Clustering.AllNodes].genome), str(net[jl.Clustering.AllNodes])) for net, mult in res]
        print({"topology": topologies, "coordination": coord, "bonds": bonds})

In [8]:
exportCif(structure)

/var/folders/h8/9hx_fyj91053ksgdzb2w03vw0000gp/T/tmpe3nch5i_.cif
{'topology': [(1, 3, 'UNKNOWN 3 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 2 0 1 1 1 2 1 1 1 1 2 2 1')], 'coordination': {('Rb', 6), ('O', 6)}, 'bonds': {('Rb', 'O')}}


In [9]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-2647057"],
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )

structure = mp_data[0].structure
bonded_struct = crystallnn.get_bonded_structure(structure)
bonded_struct

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4146: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/Users/fx/miniforge3/envs/matten/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:3939: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


Structure Graph
Structure: 
Structure Summary
Lattice
    abc : 2.428889488469988 2.428889488469988 2.428889488469988
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 11.030656874268672
      A : -1.40232 1.40232 1.40232
      B : 1.40232 -1.40232 1.40232
      C : 1.40232 1.40232 -1.40232
    pbc : True True True
PeriodicSite: Co (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
Graph: bonds
from    to  to_image    
----  ----  ------------
   0     0  (1, 1, 1)   
   0     0  (1, 0, 0)   
   0     0  (0, 0, 1)   
   0     0  (0, 1, 0)   

In [17]:
for n, site in enumerate(bonded_struct.structure.sites):
    print(site.specie)
    print(bonded_struct.get_coordination_of_site(n))
    print(len(bonded_struct.get_connected_sites(n)))

Co
4
8


In [18]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-2647057"],
        fields=["material_id", "formula", "formula_pretty", "nelements", "structure"]
    )

structure = mp_data[0].structure
bonded_struct = crystallnn.get_bonded_structure(structure)

for n, site in enumerate(bonded_struct.structure.sites):
    print(site.specie)
    print(bonded_struct.get_coordination_of_site(n)) # This is wrong
    print(len(bonded_struct.get_connected_sites(n))) # This is right

# More bug: https://github.com/materialsproject/pymatgen/issues/3888#issuecomment-2232571072

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Co
4
8


In [20]:
bonded_struct.graph.degree(0)

8